<a href="https://colab.research.google.com/github/Sankeerth55/AI-Career-Advisor-Agent/blob/main/AI_Career_Advisor_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip install google-genai

In [50]:
!pip install -q U langgraph

In [51]:
!pip install langchain_google_genai

In [52]:
!pip install tavily-python

In [53]:
!pip install -q U langchain_community

In [54]:
from google.colab import userdata
API_KEY = userdata.get('GOOGLE_API_KEY')

In [55]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=API_KEY)
response = model.invoke("What is the capital of karnataka?")
print(response.content)

The capital of Karnataka is **Bangalore**.


In [56]:
from langgraph.graph import MessageGraph, END
from langchain_core.messages import HumanMessage, AIMessage

# Node 1: Extract detailed profile
def extract_profile(state):
    resume_text = state[-1].content
    prompt = (
        f"From the following resume, extract these sections separately:\n"
        f"1. Skills\n2. Education\n3. Experience\n\nResume:\n{resume_text}"
    )
    result = model.invoke([HumanMessage(content=prompt)]).content
    return state + [AIMessage(content=result)]

# Node 2: Generate roadmap
def generate_roadmap(state):
    extracted_profile = state[-2].content
    user_goal = state[-1].content
    prompt = (
        f"Use the following profile:\n{extracted_profile}\n\n"
        f"Now create a realistic 12-month career roadmap to achieve this target:\n{user_goal}"
    )
    roadmap = model.invoke([HumanMessage(content=prompt)]).content
    return state + [AIMessage(content=roadmap)]

# Node 3: Recommend roles and projects
def recommend_jobs_projects(state):
    profile = state[-3].content  # AIMessage with extracted profile
    goal = state[-1].content     # HumanMessage with goal
    prompt = (
        f"Profile:\n{profile}\n\n"
        f"Career Aspiration:\n{goal}\n\n"
        f"Suggest 3 suitable job roles and 3 open-source projects that align well."
    )
    output = model.invoke([HumanMessage(content=prompt)]).content
    return state + [AIMessage(content=output)]

# Build the graph
graph = MessageGraph()
graph.add_node("extract_profile", extract_profile)
graph.add_node("generate_roadmap", generate_roadmap)
graph.add_node("recommend_jobs_projects", recommend_jobs_projects)

graph.set_entry_point("extract_profile")
graph.add_edge("extract_profile", "generate_roadmap")
graph.add_edge("generate_roadmap", "recommend_jobs_projects")
graph.add_edge("recommend_jobs_projects", END)

agent = graph.compile()

# Invoke the graph
response = agent.invoke([
    HumanMessage(content="""
    Candidate Resume:
    Education: Bachelor of Technology in Computer Science
    Skills: Python, LangChain, SQL
    Experience: Internship at an AI-based startup
    """),
    HumanMessage(content="My goal is to become a Data Scientist within the next year.")
])

# Print the final result
for msg in response:
    print(f"{msg.type.upper()}:\n{msg.content}\n")

/tmp/ipython-input-312507512.py:38: LangGraphDeprecatedSinceV10: MessageGraph is deprecated in LangGraph v1.0.0, to be removed in v2.0.0. Please use StateGraph with a `messages` key instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph = MessageGraph()


HUMAN:

    Candidate Resume:
    Education: Bachelor of Technology in Computer Science
    Skills: Python, LangChain, SQL
    Experience: Internship at an AI-based startup
    

HUMAN:
My goal is to become a Data Scientist within the next year.

AI:
Okay, based on the provided resume content:

**1. Skills:**

*   (None listed)

**2. Education:**

*   (None listed)

**3. Experience:**

*   (None listed)

AI:
Okay, given that the provided profile is completely blank, this roadmap will assume a **baseline starting point of very little to no prior experience or education in data science.**  It will focus on acquiring foundational skills and building a portfolio from scratch.  This is a challenging but achievable goal within 12 months, requiring dedication and consistent effort.

**Important Assumptions:**

*   **Time Commitment:**  This roadmap assumes a significant time commitment of at least 15-20 hours per week dedicated to learning and projects.  More time will accelerate progress.
* 

In [65]:
!pip install -q streamlit google-genai langchain-google-genai langgraph pyngrok

In [66]:
app_code = '''import streamlit as st
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import MessageGraph, END
from langchain_core.messages import HumanMessage, AIMessage
import time

# Page configuration
st.set_page_config(
    page_title="AI Career Roadmap Assistant",
    page_icon="🚀",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Professional Custom CSS
st.markdown("""
    <style>
    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;600;700&display=swap');

    * {
        font-family: 'Inter', sans-serif;
    }

    .main {
        padding: 1rem 2rem;
        background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);
    }

    .stButton>button {
        width: 100%;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        font-weight: 600;
        padding: 1rem 2rem;
        border-radius: 12px;
        border: none;
        box-shadow: 0 8px 20px rgba(102, 126, 234, 0.4);
        transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1);
        font-size: 1.1rem;
        letter-spacing: 0.5px;
    }

    .stButton>button:hover {
        transform: translateY(-3px);
        box-shadow: 0 12px 28px rgba(102, 126, 234, 0.5);
        background: linear-gradient(135deg, #764ba2 0%, #667eea 100%);
    }

    .hero-section {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 4rem 2rem;
        border-radius: 20px;
        margin-bottom: 2rem;
        color: white;
        text-align: center;
        box-shadow: 0 20px 60px rgba(0,0,0,0.3);
        position: relative;
        overflow: hidden;
    }

    .hero-section::before {
        content: "";
        position: absolute;
        top: -50%;
        right: -50%;
        width: 200%;
        height: 200%;
        background: radial-gradient(circle, rgba(255,255,255,0.1) 0%, transparent 70%);
        animation: rotate 20s linear infinite;
    }

    @keyframes rotate {
        from { transform: rotate(0deg); }
        to { transform: rotate(360deg); }
    }

    .hero-title {
        font-size: 3.5rem;
        font-weight: 700;
        margin-bottom: 1rem;
        position: relative;
        z-index: 1;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.2);
    }

    .hero-subtitle {
        font-size: 1.3rem;
        font-weight: 300;
        position: relative;
        z-index: 1;
        opacity: 0.95;
    }

    .card {
        background: white;
        padding: 2.5rem;
        border-radius: 16px;
        box-shadow: 0 10px 30px rgba(0,0,0,0.1);
        margin: 1.5rem 0;
        transition: transform 0.3s ease, box-shadow 0.3s ease;
        border: 1px solid rgba(102, 126, 234, 0.1);
    }

    .card:hover {
        transform: translateY(-5px);
        box-shadow: 0 15px 40px rgba(0,0,0,0.15);
    }

    .result-card {
        background: linear-gradient(145deg, #ffffff 0%, #f8f9fa 100%);
        padding: 2.5rem;
        border-radius: 16px;
        margin: 2rem 0;
        border-left: 6px solid #667eea;
        box-shadow: 0 8px 24px rgba(0,0,0,0.08);
    }

    .feature-item {
        background: white;
        padding: 1.5rem;
        border-radius: 12px;
        text-align: center;
        box-shadow: 0 4px 15px rgba(0,0,0,0.08);
        transition: all 0.3s ease;
        border-top: 4px solid #667eea;
        margin: 1rem 0;
    }

    .feature-item:hover {
        transform: translateY(-8px);
        box-shadow: 0 12px 30px rgba(102, 126, 234, 0.2);
    }

    .feature-icon {
        font-size: 2.5rem;
        margin-bottom: 1rem;
    }

    .feature-title {
        font-weight: 600;
        color: #333;
        margin-bottom: 0.5rem;
    }

    .feature-desc {
        color: #666;
        font-size: 0.9rem;
    }

    .stTextArea textarea {
        border-radius: 12px;
        border: 2px solid #e0e0e0;
        padding: 1.5rem;
        font-size: 1rem;
        transition: border-color 0.3s ease;
        background: white;
    }

    .stTextArea textarea:focus {
        border-color: #667eea;
        box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1);
    }

    .stTextInput input {
        border-radius: 12px;
        border: 2px solid #e0e0e0;
        padding: 1rem;
        font-size: 1rem;
        transition: border-color 0.3s ease;
        background: white;
    }

    .stTextInput input:focus {
        border-color: #667eea;
        box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1);
    }

    .stSelectbox select {
        border-radius: 12px;
        border: 2px solid #e0e0e0;
        padding: 0.75rem;
        background: white;
    }

    .stTabs [data-baseweb="tab-list"] {
        gap: 2rem;
        background: white;
        padding: 1rem;
        border-radius: 12px;
        box-shadow: 0 4px 15px rgba(0,0,0,0.05);
    }

    .stTabs [data-baseweb="tab"] {
        padding: 1rem 2rem;
        background: transparent;
        border-radius: 8px;
        font-weight: 600;
        color: #666;
    }

    .stTabs [aria-selected="true"] {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
    }

    .info-badge {
        display: inline-block;
        padding: 0.5rem 1rem;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border-radius: 20px;
        font-size: 0.9rem;
        font-weight: 600;
        margin: 0.5rem 0.5rem 0.5rem 0;
    }

    .success-message {
        background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%);
        color: white;
        padding: 1.5rem;
        border-radius: 12px;
        font-weight: 600;
        text-align: center;
        margin: 2rem 0;
        box-shadow: 0 8px 20px rgba(17, 153, 142, 0.3);
    }

    .section-header {
        font-size: 1.8rem;
        font-weight: 700;
        color: #333;
        margin: 2rem 0 1rem 0;
        padding-bottom: 0.5rem;
        border-bottom: 3px solid #667eea;
    }

    .subsection-header {
        font-size: 1.3rem;
        font-weight: 600;
        color: #667eea;
        margin: 1.5rem 0 1rem 0;
    }

    hr {
        border: none;
        height: 2px;
        background: linear-gradient(90deg, transparent, #667eea, transparent);
        margin: 3rem 0;
    }

    .footer {
        text-align: center;
        padding: 3rem 2rem;
        background: white;
        border-radius: 16px;
        margin-top: 3rem;
        box-shadow: 0 -5px 20px rgba(0,0,0,0.05);
    }

    .stats-container {
        display: flex;
        justify-content: space-around;
        margin: 2rem 0;
        flex-wrap: wrap;
    }

    .stat-box {
        text-align: center;
        padding: 1.5rem;
        min-width: 150px;
    }

    .stat-number {
        font-size: 2.5rem;
        font-weight: 700;
        color: #667eea;
    }

    .stat-label {
        color: #666;
        margin-top: 0.5rem;
    }
    </style>
""", unsafe_allow_html=True)

# Initialize session state
if 'results' not in st.session_state:
    st.session_state.results = None
if 'generated' not in st.session_state:
    st.session_state.generated = False

# Initialize the LangGraph agent
@st.cache_resource
def initialize_agent():
    try:
        API_KEY = userdata.get('GOOGLE_API_KEY')
        model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", api_key=API_KEY)

        def extract_profile(state):
            resume_text = state[-1].content
            prompt = (
                f"Analyze the following resume and extract information in a structured format:\\n\\n"
                f"Resume:\\n{resume_text}\\n\\n"
                f"Provide a comprehensive analysis with these sections:\\n"
                f"1. **Skills Assessment**: Categorize skills (Technical, Soft Skills, Tools)\\n"
                f"2. **Education Background**: Educational qualifications with institutions\\n"
                f"3. **Professional Experience**: Work history with key achievements\\n"
                f"4. **Strengths**: Top 3 strengths based on profile\\n"
                f"5. **Areas for Development**: Skills gaps to address\\n\\n"
                f"Format with clear markdown headers, bullet points, and emojis for visual appeal."
            )
            result = model.invoke([HumanMessage(content=prompt)]).content
            return state + [AIMessage(content=result)]

        def generate_roadmap(state):
            extracted_profile = state[-2].content
            user_goal = state[-1].content
            prompt = (
                f"Based on this candidate profile:\\n{extracted_profile}\\n\\n"
                f"Career Goal: {user_goal}\\n\\n"
                f"Create a comprehensive, actionable 12-month career transformation roadmap.\\n\\n"
                f"Structure the roadmap as follows:\\n\\n"
                f"**Phase 1: Foundation (Months 1-3)**\\n"
                f"- Learning objectives\\n"
                f"- Specific courses/certifications (with estimated hours)\\n"
                f"- Skills to master\\n"
                f"- Success metrics\\n\\n"
                f"**Phase 2: Skill Development (Months 4-6)**\\n"
                f"- Advanced topics\\n"
                f"- Hands-on projects\\n"
                f"- Portfolio building\\n"
                f"- Networking strategies\\n\\n"
                f"**Phase 3: Practical Application (Months 7-9)**\\n"
                f"- Real-world projects\\n"
                f"- Freelance opportunities\\n"
                f"- Open-source contributions\\n"
                f"- Industry certifications\\n\\n"
                f"**Phase 4: Career Transition (Months 10-12)**\\n"
                f"- Job search strategy\\n"
                f"- Interview preparation\\n"
                f"- Resume optimization\\n"
                f"- Final portfolio polish\\n\\n"
                f"Include specific resources, estimated time investment, and measurable milestones. Use emojis and formatting for clarity."
            )
            roadmap = model.invoke([HumanMessage(content=prompt)]).content
            return state + [AIMessage(content=roadmap)]

        def recommend_jobs_projects(state):
            profile = state[-3].content
            goal = state[-1].content
            prompt = (
                f"Based on this profile:\\n{profile}\\n\\n"
                f"Target Role: {goal}\\n\\n"
                f"Provide detailed recommendations:\\n\\n"
                f"**🎯 Top 5 Job Roles**\\n"
                f"For each role include:\\n"
                f"- Job title and level\\n"
                f"- Required skills (match %)\\n"
                f"- Salary range (USD)\\n"
                f"- Top companies hiring\\n"
                f"- Key responsibilities\\n\\n"
                f"**💼 Top 5 Project Recommendations**\\n"
                f"For each project include:\\n"
                f"- Project name and description\\n"
                f"- Skills you'll develop\\n"
                f"- Estimated completion time\\n"
                f"- Difficulty level\\n"
                f"- GitHub/resources link (suggest real projects)\\n"
                f"- Portfolio impact\\n\\n"
                f"**📚 Learning Resources**\\n"
                f"- Top 3 online courses\\n"
                f"- Top 3 books\\n"
                f"- Top 3 YouTube channels/podcasts\\n"
                f"- Communities to join\\n\\n"
                f"Make all recommendations specific, current, and actionable. Use emojis and clear formatting."
            )
            output = model.invoke([HumanMessage(content=prompt)]).content
            return state + [AIMessage(content=output)]

        graph = MessageGraph()
        graph.add_node("extract_profile", extract_profile)
        graph.add_node("generate_roadmap", generate_roadmap)
        graph.add_node("recommend_jobs_projects", recommend_jobs_projects)
        graph.set_entry_point("extract_profile")
        graph.add_edge("extract_profile", "generate_roadmap")
        graph.add_edge("generate_roadmap", "recommend_jobs_projects")
        graph.add_edge("recommend_jobs_projects", END)

        return graph.compile()
    except Exception as e:
        st.error(f"❌ Error initializing agent: {str(e)}")
        return None

# Hero Section
st.markdown("""
    <div class="hero-section">
        <h1 class="hero-title">🚀 AI Career Roadmap Assistant</h1>
        <p class="hero-subtitle">
            Transform Your Career Aspirations into Actionable Plans with AI-Powered Intelligence
        </p>
    </div>
""", unsafe_allow_html=True)

# Stats Section
st.markdown("""
    <div class="card">
        <div class="stats-container">
            <div class="stat-box">
                <div class="stat-number">12</div>
                <div class="stat-label">Month Roadmap</div>
            </div>
            <div class="stat-box">
                <div class="stat-number">AI</div>
                <div class="stat-label">Powered Analysis</div>
            </div>
            <div class="stat-box">
                <div class="stat-number">100%</div>
                <div class="stat-label">Personalized</div>
            </div>
        </div>
    </div>
""", unsafe_allow_html=True)

# Sidebar with features
with st.sidebar:
    st.markdown("###  Key Features")
    st.markdown("""
    <div class="feature-item">
        <div class="feature-icon"></div>
        <div class="feature-title">Smart Analysis</div>
        <div class="feature-desc">AI-powered resume parsing</div>
    </div>

    <div class="feature-item">
        <div class="feature-icon"></div>
        <div class="feature-title">Custom Roadmap</div>
        <div class="feature-desc">12-month career plan</div>
    </div>

    <div class="feature-item">
        <div class="feature-icon"></div>
        <div class="feature-title">Job Matching</div>
        <div class="feature-desc">Top role recommendations</div>
    </div>

    <div class="feature-item">
        <div class="feature-icon"></div>
        <div class="feature-title">Project Ideas</div>
        <div class="feature-desc">Portfolio-building projects</div>
    </div>

    <div class="feature-item">
        <div class="feature-icon"></div>
        <div class="feature-title">Resources</div>
        <div class="feature-desc">Curated learning materials</div>
    </div>
    """, unsafe_allow_html=True)

    st.markdown("---")
    st.markdown("###  How It Works")
    st.markdown("""
    1. **Enter** your resume details
    2. **Specify** your career goal
    3. **Choose** your timeline
    4. **Get** personalized roadmap
    5. **Download** your plan
    """)

# Main Input Section
st.markdown('<h2 class="section-header"> Your Information</h2>', unsafe_allow_html=True)

col1, col2 = st.columns([1.2, 1])

with col1:
    st.markdown('<div class="card">', unsafe_allow_html=True)
    st.markdown("###  Resume Details")
    resume_text = st.text_area(
        "Paste your resume or enter your professional information",
        height=350,
        placeholder="""Example Format:

 EDUCATION
- Bachelor of Technology in Computer Science
  XYZ University | 2019-2023 | GPA: 3.8/4.0

 TECHNICAL SKILLS
- Languages: Python, Java, SQL, JavaScript
- Frameworks: TensorFlow, PyTorch, React, Node.js
- Tools: Git, Docker, Kubernetes, AWS
- Specializations: Machine Learning, Data Analysis

 EXPERIENCE
- AI/ML Intern | TechStart Inc. | Jan 2023 - Jun 2023
  - Developed customer segmentation models (95% accuracy)
  - Built ETL pipelines processing 1M+ records daily
  - Collaborated with 5-person data science team

 PROJECTS
- Sentiment Analysis System: NLP model with 92% accuracy
- Recommendation Engine: Collaborative filtering for 10K users

 CERTIFICATIONS
- AWS Certified Solutions Architect
- Google Data Analytics Professional Certificate
""",
        help="Include education, skills, experience, projects, and certifications"
    )
    st.markdown('</div>', unsafe_allow_html=True)

with col2:
    st.markdown('<div class="card">', unsafe_allow_html=True)
    st.markdown("###  Career Preferences")

    career_goal = st.text_input(
        " Target Role",
        placeholder="e.g., Senior Data Scientist, ML Engineer, AI Researcher",
        help="Be specific about your desired position"
    )

    timeline = st.selectbox(
        " Timeline",
        ["12 months", "6 months", "18 months", "24 months"],
        help="Choose your career transition timeframe"
    )

    experience_level = st.select_slider(
        " Current Experience Level",
        options=["Beginner", "Entry-Level", "Intermediate", "Advanced", "Expert"],
        value="Intermediate"
    )

    focus_areas = st.multiselect(
        " Focus Areas (Optional)",
        ["Technical Skills", "Leadership", "Certifications", "Networking", "Portfolio", "Salary Negotiation"],
        default=["Technical Skills", "Portfolio"]
    )

    st.markdown('</div>', unsafe_allow_html=True)

# Generate Button
st.markdown("<br>", unsafe_allow_html=True)
col_btn1, col_btn2, col_btn3 = st.columns([1, 2, 1])
with col_btn2:
    generate_button = st.button(" Generate My Personalized Roadmap", use_container_width=True)

# Process and Display Results
if generate_button:
    if not resume_text or not career_goal:
        st.error(" Please provide both your resume information and career goal to continue!")
    else:
        with st.spinner(""):
            progress_bar = st.progress(0)
            status_text = st.empty()

            status_text.text(" Analyzing your profile...")
            progress_bar.progress(25)
            time.sleep(1)

            try:
                agent = initialize_agent()
                if agent:
                    status_text.text(" AI is processing your information...")
                    progress_bar.progress(50)

                    focus_str = ", ".join(focus_areas) if focus_areas else "general career development"

                    response = agent.invoke([
                        HumanMessage(content=f"Candidate Resume:\\n{resume_text}\\n\\nExperience Level: {experience_level}\\nFocus Areas: {focus_str}"),
                        HumanMessage(content=f"Career Goal: {career_goal}\\nTimeline: {timeline}\\nPlease create a detailed roadmap.")
                    ])

                    status_text.text(" Creating your roadmap...")
                    progress_bar.progress(75)
                    time.sleep(1)

                    status_text.text(" Finalizing recommendations...")
                    progress_bar.progress(100)
                    time.sleep(0.5)

                    st.session_state.results = response
                    st.session_state.generated = True

                    progress_bar.empty()
                    status_text.empty()

                    st.markdown("""
                        <div class="success-message">
                             Your Personalized Career Roadmap is Ready!
                        </div>
                    """, unsafe_allow_html=True)

            except Exception as e:
                st.error(f" An error occurred: {str(e)}")
                st.info(" Tip: Make sure your GOOGLE_API_KEY is set correctly in Colab Secrets")

# Display Results
if st.session_state.results:
    st.markdown("---")
    st.markdown('<h2 class="section-header"> Your Personalized Career Transformation Plan</h2>', unsafe_allow_html=True)

    # Quick Info Badges
    st.markdown(f"""
        <div style="margin: 1rem 0;">
            <span class="info-badge"> Goal: {career_goal}</span>
            <span class="info-badge"> Timeline: {timeline}</span>
            <span class="info-badge"> Level: {experience_level}</span>
        </div>
    """, unsafe_allow_html=True)

    tab1, tab2, tab3 = st.tabs([" Profile Analysis", " Career Roadmap", " Opportunities & Resources"])

    messages = st.session_state.results

    with tab1:
        st.markdown('<div class="result-card">', unsafe_allow_html=True)
        st.markdown('<h3 class="subsection-header"> Your Professional Profile Analysis</h3>', unsafe_allow_html=True)
        if len(messages) > 1:
            st.markdown(messages[1].content)
        else:
            st.warning("Profile analysis not available")
        st.markdown('</div>', unsafe_allow_html=True)

    with tab2:
        st.markdown('<div class="result-card">', unsafe_allow_html=True)
        st.markdown(f'<h3 class="subsection-header"> Your {timeline} Transformation Journey</h3>', unsafe_allow_html=True)
        if len(messages) > 2:
            st.markdown(messages[2].content)
        else:
            st.warning("Roadmap not available")
        st.markdown('</div>', unsafe_allow_html=True)

    with tab3:
        st.markdown('<div class="result-card">', unsafe_allow_html=True)
        st.markdown('<h3 class="subsection-header"> Curated Opportunities & Learning Resources</h3>', unsafe_allow_html=True)
        if len(messages) > 3:
            st.markdown(messages[3].content)
        else:
            st.warning("Recommendations not available")
        st.markdown('</div>', unsafe_allow_html=True)

    # Download Section
    st.markdown("---")
    st.markdown('<h2 class="section-header"> Download Your Complete Plan</h2>', unsafe_allow_html=True)

    col_dl1, col_dl2, col_dl3 = st.columns([1, 2, 1])
    with col_dl2:
        full_report = f"""
{'='*80}
AI CAREER ROADMAP ASSISTANT - PERSONALIZED REPORT
{'='*80}

Career Goal: {career_goal}
Timeline: {timeline}
Experience Level: {experience_level}
Generated: {time.strftime('%Y-%m-%d %H:%M:%S')}

{'='*80}

"""
        full_report += "\\n\\n".join([f"{'='*80}\\n{msg.content}\\n" for msg in messages if hasattr(msg, 'content')])

        st.download_button(
            label=" Download Complete Career Roadmap (TXT)",
            data=full_report,
            file_name=f"career_roadmap_{career_goal.replace(' ', '_').lower()}_{time.strftime('%Y%m%d')}.txt",
            mime="text/plain",
            use_container_width=True
        )

# Footer
st.markdown("---")
st.markdown("""
    <div class="footer">
        <h3 style="color: #667eea; margin-bottom: 1rem;"> Powered by Advanced AI</h3>
        <p style="color: #666; font-size: 1.1rem; margin-bottom: 1rem;">
            Google Gemini 2.0 Flash • LangGraph • Streamlit
        </p>
        <p style="color: #999; font-size: 0.9rem;">
            Your AI-powered career planning companion for intelligent, personalized guidance
        </p>
        <p style="color: #ccc; font-size: 0.8rem; margin-top: 1.5rem;">
             Pro Tip: Revisit and update your roadmap every quarter to track progress
        </p>
    </div>
""", unsafe_allow_html=True)
'''

# Write the app to file
with open('app.py', 'w') as f:
    f.write(app_code)

print("\n" + "="*60)
print(" Professional Streamlit app created successfully!")
print("="*60)
print("\n File created: app.py")
print(" Features: Modern UI, Professional Design, Full Functionality")
print("\n Next Step: Run CELL 3 to launch the app")
print("="*60)


 Professional Streamlit app created successfully!

 File created: app.py
 Features: Modern UI, Professional Design, Full Functionality

 Next Step: Run CELL 3 to launch the app


In [73]:
with open("app.py", "w") as f:
    f.write(app_code)

print("✅ Streamlit app written to app.py")

# ---------------- Ngrok + Streamlit Launch ----------------
from pyngrok import ngrok
import os
import time

# ⚠️ Replace this with your actual ngrok token
NGROK_TOKEN = "34SN6XksUYC7CsvljeFWcaKfPia_7yhXY3xfu8qVqkWip17wP"

# Set ngrok token
try:
    ngrok.set_auth_token(NGROK_TOKEN)
except Exception as e:
    print("❌ Error setting ngrok token!")
    print(e)
    raise

# Kill existing tunnels
ngrok.kill()

# Start Streamlit in background
print("🚀 Starting Streamlit server...")
os.system("nohup streamlit run app.py --server.port 8501 &")

# Wait for server to initialize
print("⏳ Waiting for Streamlit server to start (15s)...")
for i in range(15, 0, -1):
    print(f"   {i}...", end="\r")
    time.sleep(1)

# Create public ngrok tunnel
try:
    public_url = ngrok.connect(8501)
    print("\n🎉 Streamlit app is LIVE!")
    print("🌐 Public URL:", public_url)
    print("\n💡 Keep this Colab tab open. Reloads are faster with caching enabled in the app.")
except Exception as e:
    print("❌ Failed to create ngrok tunnel")
    print(e)

✅ Streamlit app written to app.py
🚀 Starting Streamlit server...
⏳ Waiting for Streamlit server to start (15s)...

🎉 Streamlit app is LIVE!
🌐 Public URL: NgrokTunnel: "https://3d4b749daf26.ngrok-free.app" -> "http://localhost:8501"

💡 Keep this Colab tab open. Reloads are faster with caching enabled in the app.
